In [1]:
from dotenv import load_dotenv
import openai

load_dotenv()

# list models
openai.Model.list()

<OpenAIObject list at 0x7fbd479b9180> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

In [73]:
from googlesearch import search

def search_google(query, num_results=10):
    urls = []
    for url in search(query, num_results=num_results):
        urls.append(url)
    return {
        'urls': urls
    }

def write_to_file(file_path, contents):
    print(f'called: write_to_file({file_path}, {contents})')
    try:
        with open(f'tmp/{file_path}', "w") as file:
            file.write(contents)
    except IOError:
        print(f"Error writing to {file_path}")
        return False

    return True



In [3]:
import subprocess

def execute_python_file(file_path):
  output = subprocess.check_output(
    ["python", f'{file_path}'], 
    cwd='tmp',
    universal_newlines=True
  )

  return output

In [4]:
import requests
from markdownify import markdownify as md

def fetch_url_and_markdownify(url):
  if not url.startswith("http://") and not url.startswith("https://"):
    url = "http://" + url
  try:
    response = requests.get(url)
    if response.status_code == 200:
      html_content = response.text
      markdown_content = md(html_content)
      return markdown_content
    else:
      print(f"Error: {response.status_code}")
  except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


In [85]:
from jinja2 import Template

def render_field(field_value, previous_result):
  if previous_result == None:
    return field_value

  field_template = Template(field_value)
  rendered_field = field_template.render(previous_result=previous_result)
  return rendered_field


def task_handler(task, previous_result=None):
  if task['tool'] == 'GOOGLE':
    # todo - validate params
    return search_google(
      render_field(task['query'], previous_result)
    )
  elif task['tool'] == 'FILEWRITER':
    # todo - validate params
    return write_to_file(
      render_field(task['file_path'], previous_result),
      render_field(task['contents'], previous_result)
    )
  elif task['tool'] == 'FETCH_URL':
    # todo - validate params
    return fetch_url_and_markdownify(
      render_field(task['url'], previous_result)
    )
  
  elif task['tool'] == 'EXECUTE_PYTHON_FILE':
    return execute_python_file(
      render_field(task['file_path'], previous_result)
    )
  
  else:
    print("No tool matched")
    

In [86]:
import json

def marshall_tasklist(raw_completion):
  trimmed_text = raw_completion.split('<!-- RESPONSE_START -->', 1)[-1]
  trimmed_text = trimmed_text.split('<!-- RESPONSE_END -->', 1)[0]

  # todo - JSON fixing logic if needed
  try:
    return json.loads(trimmed_text)
  except:
    print("Error parsing JSON", trimmed_text)
    return None

In [98]:
from jinja2 import Environment, FileSystemLoader


env = Environment(loader=FileSystemLoader("./templates"))
template = env.get_template("prompt.txt")


def generate_response(question):
  rendered_prompt = template.render(message=question)
  completion = openai.Completion.create(
    model="davinci", 
    prompt=rendered_prompt,
    max_tokens=500,
    temperature=.5,
    stop="<!-- RESPONSE_END -->",
    best_of=5
  )

  return completion.choices[0].text


In [99]:

def process_message(message):
  raw_task = generate_response(message)
  print("=== Full Raw Tasklist ===\n", raw_task, "\n\n")

  tasklist = marshall_tasklist(raw_task)

  previous_result = None
  for task in tasklist:
    print("=== TASK ===\n", task)
    previous_result = task_handler(task, previous_result=previous_result)
    print("=== TASK_RESULT ===\n", previous_result)


In [101]:
import os

tmp_dir = './tmp'
def clear_tmp_dir():
  files = os.listdir(tmp_dir)

  # Iterate over the files and delete them
  for file in files:
    file_path = os.path.join(tmp_dir, file)
    if os.path.isfile(file_path):
      os.remove(file_path)

# message = "Fetch the first result from Google search for 'What is AutGPT' and save it in a text file"
# message = "Generate lorum ipsum text and write it into temp.txt"
# message = "fetch www.google.com"
# message = "Generate python code that prints hello world and put it into temp.py"

# message = "Create a file called temp.txt and write lorem ipsome into it. Create python script that reads the file and prints it. Execute the python script."

# message = "Search on google for 'What is AutoGPT' and then put the output into a temp.txt file. Then print out the contents of temp.txt using a python script"

clear_tmp_dir()
process_message(message)

=== Full Raw Tasklist ===
 
<!-- RESPONSE_START -->
[
  {
    "tool": "FILEWRITER",
    "file_path": "temp.txt",
    "contents": "lorem ipsum"
  },
  {
    "tool": "EXECUTE_PYTHON_FILE",
    "file_path": "test.py"
  }
]
 


=== TASK ===
 {'tool': 'FILEWRITER', 'file_path': 'temp.txt', 'contents': 'lorem ipsum'}
called: write_to_file(temp.txt, lorem ipsum)
=== TASK_RESULT ===
 True
=== TASK ===
 {'tool': 'EXECUTE_PYTHON_FILE', 'file_path': 'test.py'}


Traceback (most recent call last):
  File "/Users/alprielse/src/sandboxed/2023_llm-experiments/.venv/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 3489, in <module>
    main()
  File "/Users/alprielse/src/sandboxed/2023_llm-experiments/.venv/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 3482, in main
    globals = debugger.run(setup['file'], None, None, is_module)
  File "/Users/alprielse/src/sandboxed/2023_llm-experiments/.venv/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2510, in run
    return self._exec(is_module, entry_point_fn, module_name, file, globals, locals)
  File "/Users/alprielse/src/sandboxed/2023_llm-experiments/.venv/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2517, in _exec
    globals = pydevd_runpy.run_path(file, globals, '__main__')
  File "/Users/alprielse/src/sandboxed/2023_llm-experiments/.venv/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pyd

CalledProcessError: Command '['python', 'test.py']' returned non-zero exit status 1.